In [1]:
import numpy as np
import pandas as pd
import pickle
from xgboost import XGBClassifier
from xgboost import plot_importance

import torch
print(torch.__version__)

import torch.optim as optim
import torch.utils.data as data_utils

from torch.utils.tensorboard import SummaryWriter
from pytorch_lightning.metrics import Accuracy

from network import DeepFMNet
from data_loader import CustomDataset

1.8.1+cu101


In [2]:
EPOCHS = 500
EMBEDDING_SIZE = 5
BATCH_SIZE = 512
NROF_LAYERS = 3
NROF_NEURONS = 50
DEEP_OUTPUT_SIZE = 50
NROF_OUT_CLASSES = 1
LEARNING_RATE = 3e-4
TRAIN_PATH = '../data/train_adult.pickle'
VALID_PATH = '../data/valid_adult.pickle'

Посмотрим на датасет, может из-за него у нас появляется NaN в лоссе

In [3]:
with open('../data/train_adult.pickle', 'rb') as f:
    df, nrof_emb_categories, unique_categories = pickle.load(f)

In [28]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,native-country,salary,workclass_cat,education_cat,marital-status_cat,occupation_cat,relationship_cat,race_cat,sex_cat,native-country_cat
16422,58,Private,72812,10th,6.0,Never-married,Other-service,Not-in-family,White,Male,...,United-States,<50k,4,0,4,8,1,4,1,39
16897,21,Without-pay,232719,HS-grad,9.0,Never-married,Craft-repair,Own-child,Black,Male,...,United-States,<50k,8,11,4,3,3,2,1,39
18910,18,Self-emp-inc,117372,11th,7.0,Never-married,Sales,Own-child,White,Male,...,United-States,<50k,5,1,4,12,3,4,1,39
30095,39,Private,187098,Prof-school,15.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,...,United-States,>=50k,4,14,2,4,5,4,0,39
19533,32,Federal-gov,191385,Assoc-acdm,12.0,Divorced,Protective-serv,Not-in-family,White,Male,...,United-States,<50k,1,7,0,11,1,4,1,39


In [4]:
nrof_emb_categories

{'workclass': 9,
 'education': 16,
 'marital-status': 7,
 'occupation': 16,
 'relationship': 6,
 'race': 5,
 'sex': 2,
 'native-country': 42}

In [5]:
unique_categories

{'workclass': array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype='<U17'),
 'education': array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype='<U13'),
 'marital-status': array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
        ' Married-spouse-absent', ' Never-married', ' Separated',
        ' Widowed'], dtype='<U22'),
 'occupation': array([' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair',
        ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners',
        ' Machine-op-inspct', ' Other-service', ' Priv-house-serv',
        ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support',
        ' Transport-moving', 'nan'], dtype='<U18'),
 'relationship': array([' 

In [6]:
df.isnull().sum()

age                     0
workclass               0
fnlwgt                  0
education               0
education-num         392
marital-status          0
occupation            397
relationship            0
race                    0
sex                     0
capital-gain            0
capital-loss            0
hours-per-week          0
native-country          0
salary                  0
workclass_cat           0
education_cat           0
marital-status_cat      0
occupation_cat          0
relationship_cat        0
race_cat                0
sex_cat                 0
native-country_cat      0
dtype: int64

Все понятно, скорее всего пропущенные значения и послужили некорректому расчету loss'а

In [7]:
df['occupation'].unique()

array([' Exec-managerial', ' Adm-clerical', ' Other-service',
       ' Prof-specialty', ' Sales', ' ?', ' Transport-moving',
       ' Machine-op-inspct', ' Tech-support', ' Handlers-cleaners',
       ' Protective-serv', ' Craft-repair', ' Farming-fishing', nan,
       ' Priv-house-serv', ' Armed-Forces'], dtype=object)

In [8]:
df['education-num'].unique()

array([ 9., 10., 13., 12., 15.,  6.,  5., 14., nan,  4., 11.,  7.,  3.,
       16.,  1.,  8.,  2.])

В таком случае заменим пропущенные значения, исходя из здравого смысла

In [9]:
df['education-num'] = df['education-num'].fillna(0.)

In [10]:
df['occupation'] = df['occupation'].fillna('NaN')

Сораняем наш тренироавочный датасет в pickle формат

In [12]:
with open('../data/train_adult.pickle', 'wb') as f:
    pickle.dump([df, nrof_emb_categories, unique_categories], f)

Аналогичные манипуляции проводим с валидационным датасетом

In [13]:
with open('../data/valid_adult.pickle', 'rb') as f:
    df, nrof_emb_categories, unique_categories = pickle.load(f)

In [14]:
df.isnull().sum()

age                     0
workclass               0
fnlwgt                  0
education               0
education-num          95
marital-status          0
occupation            115
relationship            0
race                    0
sex                     0
capital-gain            0
capital-loss            0
hours-per-week          0
native-country          0
salary                  0
workclass_cat           0
education_cat           0
marital-status_cat      0
occupation_cat          0
relationship_cat        0
race_cat                0
sex_cat                 0
native-country_cat      0
dtype: int64

In [15]:
df['education-num'] = df['education-num'].fillna(0.)
df['occupation'] = df['occupation'].fillna('NaN')

In [16]:
# дамп в pickle
with open('../data/valid_adult.pickle', 'wb') as f:
    pickle.dump([df, nrof_emb_categories, unique_categories], f)

In [8]:
class DeepFM:
    def __init__(self):
        self.train_dataset = CustomDataset(TRAIN_PATH)
        self.train_loader = data_utils.DataLoader(dataset=self.train_dataset,
                                                  batch_size=BATCH_SIZE, shuffle=True)
        self.valid_dataset = CustomDataset(VALID_PATH)
        self.valid_loader = data_utils.DataLoader(dataset=self.valid_dataset,
                                                  batch_size=BATCH_SIZE, shuffle=True)

        self.build_model()

        self.log_params()

        self.train_writer = SummaryWriter('./logs/train')
        self.valid_writer = SummaryWriter('./logs/valid')

        return

    def build_model(self):
        self.network = DeepFMNet(nrof_cat=self.train_dataset.nrof_emb_categories, emb_dim=EMBEDDING_SIZE,
                                 emb_columns=self.train_dataset.embedding_columns,
                                 numeric_columns=self.train_dataset.numeric_columns,
                                 nrof_layers=NROF_LAYERS, nrof_neurons=NROF_NEURONS,
                                 output_size=DEEP_OUTPUT_SIZE,
                                 nrof_out_classes=NROF_OUT_CLASSES)

        self.loss = torch.nn.BCEWithLogitsLoss()
        self.accuracy = Accuracy()
        self.optimizer = optim.Adam(self.network.parameters(), lr=LEARNING_RATE)

        return

    def log_params(self):
        return

    def load_model(self, restore_path=''):
        if restore_path == '':
            self.step = 0
        else:
            pass

        return

    def run_train(self):
        print('Run train ...')

        self.load_model()

        for epoch in range(EPOCHS):
            self.network.train()

            for features, label in self.train_loader:
                # Reset gradients
                self.optimizer.zero_grad()

                output = self.network(features)
                # Calculate error and backpropagate
                loss = self.loss(output, label)

                output = torch.sigmoid(output)

                loss.backward()

                acc = self.accuracy(output, label.long()).item()

                # Update weights with gradients
                self.optimizer.step()

                self.train_writer.add_scalar('CrossEntropyLoss', loss, self.step)
                self.train_writer.add_scalar('Accuracy', acc, self.step)

                self.step += 1

                if self.step % 50 == 0:
                    print('EPOCH %d STEP %d : train_loss: %f train_acc: %f' %
                          (epoch, self.step, loss.item(), acc))

#             self.train_writer.add_histogram('hidden_layer', self.network.linear1.weight.data, self.step)

            # Run validation
            running_loss = []
            valid_scores = []
            valid_labels = []
            self.network.eval()
            with torch.no_grad():
                for features, label in self.valid_loader:
                    output = self.network(features)
                    # Calculate error and backpropagate
                    loss = self.loss(output, label)

                    running_loss.append(loss.item())

                    valid_scores.extend((torch.sigmoid(output)>0.5).float())
                    valid_labels.extend(label.long())
            
            valid_accuracy = self.accuracy(torch.tensor(valid_scores), torch.tensor(valid_labels)).item()

            self.valid_writer.add_scalar('CrossEntropyLoss', np.mean(running_loss), self.step)
            self.valid_writer.add_scalar('Accuracy', valid_accuracy, self.step)

            print('EPOCH %d : valid_loss: %f valid_acc: %f' % (epoch, np.mean(running_loss), valid_accuracy))

        return

In [9]:
deep_fm = DeepFM()
deep_fm.run_train()

Run train ...
EPOCH 0 STEP 50 : train_loss: 0.635479 train_acc: 0.716797
EPOCH 0 : valid_loss: 0.571243 valid_acc: 0.737295
EPOCH 1 STEP 100 : train_loss: 0.437494 train_acc: 0.792969
EPOCH 1 : valid_loss: 0.378073 valid_acc: 0.833257
EPOCH 2 STEP 150 : train_loss: 0.366967 train_acc: 0.818359
EPOCH 2 : valid_loss: 0.344392 valid_acc: 0.845540
EPOCH 3 STEP 200 : train_loss: 0.328503 train_acc: 0.851562
EPOCH 3 : valid_loss: 0.329548 valid_acc: 0.851067
EPOCH 4 STEP 250 : train_loss: 0.309694 train_acc: 0.871094
EPOCH 4 : valid_loss: 0.324720 valid_acc: 0.851528
EPOCH 5 STEP 300 : train_loss: 0.295606 train_acc: 0.876953
EPOCH 5 : valid_loss: 0.322449 valid_acc: 0.853370
EPOCH 6 STEP 350 : train_loss: 0.309161 train_acc: 0.857422
EPOCH 6 : valid_loss: 0.321145 valid_acc: 0.852756
EPOCH 7 STEP 400 : train_loss: 0.323807 train_acc: 0.843750
EPOCH 7 : valid_loss: 0.318305 valid_acc: 0.853217
EPOCH 8 STEP 450 : train_loss: 0.289859 train_acc: 0.857422
EPOCH 8 : valid_loss: 0.319298 valid_ac

KeyboardInterrupt: 